In [1]:
!pip install llamaparse vertexai python-dotenv pyspellchecker huggingface-hub llama-index transformers sentence-transformers llama-index-llms-huggingface llama-index-embeddings-huggingface pdfplumber llama-index-llms-openrouter llama-index-retrievers-bm25 tabula-py  jpype1 pystemmer

!apt-get install -y poppler-utils tesseract-ocr  # Ensure poppler and Tesseract are installed
!pip install pdf2image pytesseract  # Install required Python libraries


ERROR: Could not find a version that satisfies the requirement llamaparse (from versions: none)
ERROR: No matching distribution found for llamaparse
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


## 1. Setup & Configuration
In this section, we set up the necessary dependencies, including third-party libraries and API keys.
We also configure the language models (LLMs) and embedding models used throughout the tutorial.

Key Steps:
- Import required Python libraries.
- Load API keys securely from environment variables.
- Initialize the OpenRouter LLMs for both querying and evaluation.
- Set up the HuggingFace embedding model for text representation.
- Apply `nest_asyncio` to handle event loop issues in Jupyter environments.


In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Suppresses TensorFlow warnings

from dotenv import load_dotenv
import Stemmer
import nest_asyncio
import tabula
import pandas as pd
from dotenv import load_dotenv
from llama_index.core import Document
from llama_index.core import (SimpleDirectoryReader, VectorStoreIndex, Settings)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter
import asyncio
from llama_parse import LlamaParse
import re
from spellchecker import SpellChecker
from pdf2image import convert_from_path
import pytesseract
from PIL import Image


load_dotenv()  # Load environment variables from .env file

# ✅ Load API Key Securely (No Hardcoding!)
api_key = os.getenv("OPENROUTER_API_KEY")
os.environ["LLAMA_CLOUD_API_KEY"] = # KEY
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = # JSON Key file

if api_key:
    print("✅ API Key Loaded Successfully:", api_key[:5] + "..." + api_key[-5:])
else:
    print("⚠️ API Key is missing! Check your .env file.")


# ✅ Initialize OpenRouter LLM
llm = OpenRouter(api_key=api_key, model="mistralai/mistral-7b-instruct", max_tokens=512, context_window=4096)
Judge_llm = OpenRouter(api_key=api_key, model="qwen/qwen-turbo", max_tokens=512, context_window=4096)
Settings.llm = llm

# ✅ Apply nest_asyncio to fix event loop issues in Jupyter
nest_asyncio.apply()

# ✅ Set up embedding model
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
Settings.embed_model = embed_model


✅ API Key Loaded Successfully: sk-or...24b1e


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## 2. Document Loading & Preprocessing
Here, we load the WebMD document from a PDF file and process it for retrieval.

Key Steps:
- Read the WebMD PDF and extract its textual content.
- Define an ingestion pipeline to preprocess text by splitting it into manageable chunks and applying embeddings.


In [3]:
# # ✅ Load Course Syllabus Document
pdf_path = "./WebMD.pdf"

# set up parser
parser = LlamaParse(
    result_type="markdown"
)

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=[pdf_path], file_extractor=file_extractor).load_data()

# Convert table data into an additional Document
tables = tabula.read_pdf(pdf_path, pages="all")
table_docs = [df.to_markdown(index=False) for df in tables]
all_tables_text = "\n\n".join(table_docs)
document_from_tables = Document(text=all_tables_text)

Started parsing the file under job_id e181a395-4172-43c6-9eb2-3d1733fe67be


# Task 5

**Noise reduction in OCR retrieval**

In [4]:
# Step 1: Basic cleanup
def clean_ocr_text(ocr_text):
    # Removing unwanted control characters (e.g., \x0c, \n) which I observed in image_text
    # Sample of image_text is provided in the report submitted.
    cleaned_text = re.sub(r'[\x0c\n\r]+', ' ', ocr_text)
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)  # Remove non-word characters

    # Removing extra spaces between words
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    # Correcting common OCR errors (e.g., broken words like CoM -> Com, fiX -> fix)
    cleaned_text = re.sub(r'\b(\w)([A-Z])\b', r'\1\2', cleaned_text)  # Fix broken words

    # Removing unnecessary details (e.g., WebMD references, phone numbers, etc.)
    cleaned_text = re.sub(r'WebMD|source:|GETTY IMAGES|^\d{3}-\d{3}-\d{4}', '', cleaned_text)

    # Strip leading and trailing spaces
    cleaned_text = cleaned_text.strip()
    cleaned_text = cleaned_text.lower()

    return cleaned_text


# Step 2: Correcting common OCR errors
def correct_ocr_errors(ocr_text):
    ocr_corrections = {
        '0': 'O',  # Replacing '0' with 'O'
        '1': 'I',  # Replacing '1' with 'I'
        'l': 'I',  # Replacing 'l' with 'I'
        'I': 'i',  # Replacing 'I' with 'i'
    }
    for error, correction in ocr_corrections.items():
        ocr_text = ocr_text.replace(error, correction)

    return ocr_text


# Step 3: Spell-checking and correction
def spell_check_text(ocr_text):
    print(f"\nocr text: {ocr_text}")
    spell = SpellChecker()
    words = ocr_text.split()

    # Identify misspelled words
    misspelled = spell.unknown(words)
    print(f"misspelled: {misspelled}")
    # Correct the misspelled words
    corrected_words = []
    for word in words:
        # Check if the word is in misspelled words and correct it
        if word in misspelled:
            correction = spell.correction(word)
            # Ensure the correction is not None
            if correction:
                corrected_words.append(correction)
            else:
                corrected_words.append(word)  # Append the original word if no correction found
        else:
            corrected_words.append(word)

    # Join the corrected words into a single string
    corrected_text = ' '.join(corrected_words)
    print(f"Cleaned: {corrected_text}")
    return corrected_text


# Step 4: Full OCR post-processing function
def process_ocr_text(ocr_text):
    # Ensure the text is not None and is a string
    if not ocr_text or not isinstance(ocr_text, str):
        return ''  # Return an empty string if the text is invalid

    # Clean the text (remove unwanted characters and extra spaces)
    ocr_text = clean_ocr_text(ocr_text)

    # Correct common OCR errors
    ocr_text = correct_ocr_errors(ocr_text)

    # Perform spell-checking
    ocr_text = spell_check_text(ocr_text)

    return ocr_text


# cleaned_text = []
# for text in image_text:
#   if text:
#     cleaned_text.append(process_ocr_text(text))


In [5]:
# Set correct Tesseract path (default on Colab)
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

# Convert PDF pages to images
images = convert_from_path("WebMD.pdf", poppler_path="/usr/bin/")

# Extract text from images
image_text = []
for img in images:
    img = img.convert("RGB")
    extracted_text = pytesseract.image_to_string(img, lang="eng")
    print(f"\n Extracted Text: {extracted_text}")
    image_text.append(process_ocr_text(extracted_text))

# ✅ Convert extracted text to Document format
image_text_docs = [Document(text=text) for text in image_text]


 Extracted Text:  

REAL STORIES

 
 
 
  
  
 
   

A look inside
your head
and more

joys FX- Be]

CoM A Mil-lomol ait -le3
elt emai:


ocr text: reai stories a iook inside your head and more joys fx be com a miiiomoi ait ie3 eit emai
misspelled: {'fx', 'reai', 'emai', 'eit', 'iook', 'ie3', 'miiiomoi'}
Cleaned: real stories a look inside your head and more joys fix be com a miiiomoi ait ier it email

 Extracted Text: WebMD | cHRONIC MIGRAINE CONTENTS

 

 

NEWS

and more

 

 

The latest research Being my own advocate

HEALTH SMARTS VIEWPOINT

A look inside your head Here’s how | help others help me

TREATMENT SMARTS LIVING HEALTHY

anage your chronic migraine
‘

INSIDE THE ISSUE

FIRST PERSON

through chronic migraine

Prioritize lifestyle changes

   

 

 

WebMD Real Stories is not responsible for advertising claims. WebMD Real Stories (ISSN 1553-9946) is published by WebMD LLC and may not be reproduced in whole or in part without written permission
of WebMD LLC. All rights r

In [6]:
# cleaned text
image_text

['real stories a look inside your head and more joys fix be com a miiiomoi ait ier it email',
 'chronic migraine contents news and more the latest research being my own advocate health smarts viewpoint a look inside your head heres how help others help me treatment smarts living healthy manage your chronic migraine inside the issue first person through chronic migraine prioritize lifestyle changes real stories is not responsible for advertising claims real stories in i5539946 is published by vic and may not be reproduced in whole or in part without written permission of vic air rights reserved air editorial content is reviewed by our boardcertified physicians is for informational purposes only and is not intended to be a substitute for professional medical advice diagnosis or treatment does not endorse any specific product service or treatment always seek the advice of your health care provider with any questions regarding a medical condition and never disregard professional medical ad

In [7]:
# Combine original + table doc
documents = documents + [document_from_tables] + image_text_docs

In [8]:
# ✅ Create the pipeline with transformations
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=[SentenceSplitter(chunk_size=1000, chunk_overlap=100), embed_model])

# Run the pipeline
nodes = pipeline.run(documents=documents)
print(nodes[:3])  # Print sample processed nodes

[TextNode(id_='62173ad0-16df-4964-b050-5146b493c046', embedding=[-0.034711845219135284, -0.053511325269937515, -0.01890306919813156, 0.03488024324178696, -0.055502135306596756, 0.04149353504180908, 0.06009526923298836, 0.00403983797878027, 0.038006119430065155, -0.03648754954338074, 0.04661840572953224, -0.03944697976112366, 0.020265555009245872, 0.059308238327503204, -0.028348328545689583, 0.03362327814102173, -0.06621839106082916, -0.0826292484998703, 0.06982559710741043, 0.02495506964623928, -0.114281564950943, 0.052930839359760284, -0.013911866582930088, 0.06388604640960693, -0.014764179475605488, 0.043306026607751846, -0.06454232335090637, 0.033830560743808746, -0.008696266449987888, -0.019472861662507057, 0.04228026792407036, 0.049207963049411774, -0.06821101158857346, 0.07926555722951889, 0.0391484759747982, 0.01473008468747139, -0.08382292091846466, 0.002150041051208973, -0.008015533909201622, 0.06771250069141388, -0.018836038187146187, -0.014044856652617455, -0.038659140467643

In [9]:
from llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)


# ✅ Hierarchical Parsing
hierarchical_parser = HierarchicalNodeParser.from_defaults()
heirarchical_nodes = hierarchical_parser.get_nodes_from_documents(documents)
print(heirarchical_nodes[:3])

[TextNode(id_='38a94c94-803d-4cab-a474-a96ba5554c9e', embedding=None, metadata={'file_path': 'WebMD.pdf', 'file_name': 'WebMD.pdf', 'file_type': 'application/pdf', 'file_size': 10299888, 'creation_date': '2025-03-01', 'last_modified_date': '2025-03-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d1902d46-9f05-4153-8142-71b9dc3d67f1', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'WebMD.pdf', 'file_name': 'WebMD.pdf', 'file_type': 'application/pdf', 'file_size': 10299888, 'creation_date': '2025-03-01', 'last_modified_date': '2025-03-01'}, hash='d8d6428cfaa2d546a6a525e8cf19f0730c77148575b731f7e9fbcf637be660fe'), <NodeRelationship.CHILD: '5'>: [RelatedNodeInfo(node_id='4bf94478-d470-486d-8

## 3. Indexing & Retrieval
Once the document is processed, we create multiple retrievers for efficient information retrieval.

Key Steps:
- Build a `VectorStoreIndex` from the preprocessed document chunks.
- Implement different retrieval methods:
  - **Base Retriever:** Retrieves the most relevant document chunk.
  - **AutoMerging Retriever:** Aggregates multiple related chunks before returning results.
  - **BM25 Retriever:** Uses term frequency-based ranking (a traditional information retrieval method).
  - **Hybrid Fusion Retriever:** (Task 4)
- Define retrieval hyperparameters, such as similarity threshold and number of top results to return.


In [10]:
# ✅ Create Vector Index and Query Engine
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine()

# ✅ Create base retrievers
base_retriever = index.as_retriever(similarity_top_k=1)

from llama_index.core.retrievers import AutoMergingRetriever
auto_base_retriever = index.as_retriever(similarity_top_k=3)
auto_merging_retriever = AutoMergingRetriever(auto_base_retriever, index.storage_context)

from llama_index.retrievers.bm25 import BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2, stemmer=Stemmer.Stemmer("english"), language="english")

# ❓ Create Hybrid Fusion Retriever (Task 4)
from llama_index.core.retrievers import QueryFusionRetriever

base_retriever_hybrid = index.as_retriever(similarity_top_k=5)  # Higher top-k for fusion
bm25_retriever_hybrid = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=5, stemmer=Stemmer.Stemmer("english"), language="english")

hybrid_retriever = QueryFusionRetriever(
    retrievers=[base_retriever_hybrid, bm25_retriever_hybrid],
    retriever_weights=[0.6, 0.4],
    )

DEBUG:bm25s:Building index from IDs objects
DEBUG:bm25s:Building index from IDs objects


## 4. Query Engines & Evaluation Setup
In this section, we configure the query engines and set up evaluation metrics to assess retriever performance.

Key Steps:
- Instantiate query engines for each retrieval method to allow direct querying.
- Define evaluation models to measure:
  - **Faithfulness:** Whether the retrieved information is accurate and grounded in the original document.
  - **Relevancy:** Whether the retrieved information is relevant to the query.
- Configure different retriever evaluators to assess retrieval effectiveness using metrics such as:
  - Mean Reciprocal Rank (MRR)
  - Hit Rate
  - Precision
  - Recall


In [11]:
# ✅ Create query engines
from llama_index.core.query_engine import RetrieverQueryEngine

base_query_engine = RetrieverQueryEngine.from_args(base_retriever)
auto_query_engine = RetrieverQueryEngine.from_args(auto_merging_retriever)
bm25_query_engine = RetrieverQueryEngine.from_args(bm25_retriever)

# ✅ Create hybrid query engine
hybrid_query_engine = RetrieverQueryEngine.from_args(hybrid_retriever)


# ✅ Initialize Evaluators
from llama_index.core.evaluation import FaithfulnessEvaluator, RelevancyEvaluator, RetrieverEvaluator

faithfulness_evaluator = FaithfulnessEvaluator(llm=Judge_llm)
relevancy_evaluator = RelevancyEvaluator(llm=Judge_llm)

# ✅ Define retriever evaluators
base_retriever_evaluator = RetrieverEvaluator.from_metric_names(["mrr", "hit_rate", "precision", "recall"], retriever=base_retriever)
auto_retriever_evaluator = RetrieverEvaluator.from_metric_names(["mrr", "hit_rate", "precision", "recall"], retriever=auto_merging_retriever)
bm25_retriever_evaluator = RetrieverEvaluator.from_metric_names(["mrr", "hit_rate", "precision", "recall"], retriever=bm25_retriever)

# ✅ Define retriever evaluator for hybrid approach
hybrid_retriever_evaluator = RetrieverEvaluator.from_metric_names(["mrr", "hit_rate", "precision", "recall"], retriever=hybrid_retriever)



## 5. Preparing Evaluation Questions & Display Functions

In this section, we define a set of evaluation questions to systematically assess the performance of different retrieval methods. Additionally, we create utility functions for displaying results in a structured and readable format.

Key Steps:
- Define a set of test queries (e.g., syllabus-related questions).
- Generate question-context pairs for automated evaluation.

To make evaluation results more interpretable, we define:
- **`displayify_df(df)`** – A helper function to format and display DataFrames neatly in Jupyter notebooks.
- **`display_retriever_eval_results(name, eval_results)`** – Computes and prints key retrieval evaluation metrics.

In [12]:
# ✅ Use Evaluation Questions
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        eval_questions.append(line.strip())

from llama_index.core.evaluation import generate_question_context_pairs
qa_dataset = generate_question_context_pairs(nodes=nodes, llm=llm, num_questions_per_chunk=1)


# ✅ Pretty Display Function
def displayify_df(df):
    """For pretty displaying DataFrame in a notebook."""
    display_df = df.style.set_properties(
        **{
            "inline-size": "300px",
            "overflow-wrap": "break-word",
        }
    )
    display(display_df)

# Helper to display retrieval metrics
def display_retriever_eval_results(name, eval_results):
    """Build a small DataFrame summarizing retrieval metrics across queries."""
    print(f"=== {name} ===")
    metric_dicts = [res.metric_vals_dict for res in eval_results]
    if not metric_dicts:
        print("No retriever metrics found!")
        return
    df = pd.DataFrame(metric_dicts)
    #displayify_df(df)
    print("Mean:\n", df.mean(numeric_only=True), "\n")

100%|██████████| 19/19 [01:16<00:00,  4.05s/it]


## 6. Running Evaluations & Comparing Retrievers
Finally, we conduct retrieval evaluations and compare the effectiveness of different retrieval strategies.

Key Steps:
- Execute retrieval on all retrievers (Base, AutoMerging, BM25, Hybrid Fusion).
- Assess each retriever's faithfulness and relevancy scores.
- Compare retrieval results by averaging the evaluation metrics.
- Display results in tabular format for easy comparison.

This step helps determine which retrieval method performs best for a given dataset.


In [13]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part


PROJECT_ID = "my-vertexai-project-435101"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
parameters = {
    "temperature": 0,
    "max_output_tokens": 256,
    "top_p": 0.95,
    "top_k": 40,
}

def paraphrase_query(query):
  model = GenerativeModel("gemini-1.5-flash-002")
  response = model.generate_content(
    f"Generate 1 different way to ask the following question:\n{query}. Only give questions.",
  )
  print(response.candidates[0].content.text)
  print(f"Questions generated: {response.candidates[0].content.text}")
  return response.candidates[0].content.text



# ✅ Modify Evaluation to Include Fusion Retriever
async def run_evaluation():
    eval_results = []

    for para_query in eval_questions:
      paraphrased_queries = []
      paraphrased_queries.append(para_query)
      paraphrased_queries.append(paraphrase_query(para_query))

      for query in paraphrased_queries:
        # Retrieve responses from all four retrievers
        base_response = base_query_engine.query(query)
        auto_response = auto_query_engine.query(query)
        bm25_response = bm25_query_engine.query(query)
        hybrid_response = hybrid_query_engine.query(query)

        # Extract text from responses
        base_text = base_response.response
        auto_text = auto_response.response
        bm25_text = bm25_response.response
        hybrid_text = hybrid_response.response

        base_contexts = "\n".join([node.get_content() for node in base_response.source_nodes])
        auto_contexts = "\n".join([node.get_content() for node in auto_response.source_nodes])
        bm25_contexts = "\n".join([node.get_content() for node in bm25_response.source_nodes])
        hybrid_contexts = "\n".join([node.get_content() for node in hybrid_response.source_nodes])

        # Evaluate Faithfulness & Relevancy for each retriever
        base_faithfulness = faithfulness_evaluator.evaluate_response(response=base_response)
        auto_faithfulness = faithfulness_evaluator.evaluate_response(response=auto_response)
        bm25_faithfulness = faithfulness_evaluator.evaluate_response(response=bm25_response)
        hybrid_faithfulness = faithfulness_evaluator.evaluate_response(response=hybrid_response)

        base_relevancy = relevancy_evaluator.evaluate_response(query=query, response=base_response)
        auto_relevancy = relevancy_evaluator.evaluate_response(query=query, response=auto_response)
        bm25_relevancy = relevancy_evaluator.evaluate_response(query=query, response=bm25_response)
        hybrid_relevancy = relevancy_evaluator.evaluate_response(query=query, response=hybrid_response)

        eval_results.append({
            "Query": query,
            "Base Response": base_text,
            "Auto-Merged Response": auto_text,
            "BM25 Response": bm25_text,
            "Hybrid Response": hybrid_text,
            "Base Context": "".join(base_contexts[:100]) + "... " + "".join(base_contexts[-100:]),
            "Auto Context": "".join(auto_contexts[:100]) + "... " + "".join(auto_contexts[-100:]),
            "BM25 Context": "".join(bm25_contexts[:100]) + "... " + "".join(bm25_contexts[-100:]),
            "Hybrid Context": "".join(hybrid_contexts[:100]) + "... " + "".join(hybrid_contexts[-100:]),
            "Base Faithfulness": base_faithfulness.score,
            "Auto Faithfulness": auto_faithfulness.score,
            "BM25 Faithfulness": bm25_faithfulness.score,
            "Hybrid Faithfulness": hybrid_faithfulness.score,
            "Base Relevancy": base_relevancy.score,
            "Auto Relevancy": auto_relevancy.score,
            "BM25 Relevancy": bm25_relevancy.score,
            "Hybrid Relevancy": hybrid_relevancy.score,
        })

    df = pd.DataFrame(eval_results)

    # Compute means for comparison
    print("\n✅ Faithfulness Comparison")
    print(f"Base Retriever: {df['Base Faithfulness'].mean():.4f}")
    print(f"Auto-Merging Retriever: {df['Auto Faithfulness'].mean():.4f}")
    print(f"BM25 Retriever: {df['BM25 Faithfulness'].mean():.4f}")
    print(f"Hybrid Retriever: {df['Hybrid Faithfulness'].mean():.4f}")

    print("\n✅ Relevancy Comparison")
    print(f"Base Retriever: {df['Base Relevancy'].mean():.4f}")
    print(f"Auto-Merging Retriever: {df['Auto Relevancy'].mean():.4f}")
    print(f"BM25 Retriever: {df['BM25 Relevancy'].mean():.4f}")
    print(f"Hybrid Retriever: {df['Hybrid Relevancy'].mean():.4f}")

    # Evaluate all retrievers on the QA dataset
    base_eval_results = await base_retriever_evaluator.aevaluate_dataset(qa_dataset)
    auto_eval_results = await auto_retriever_evaluator.aevaluate_dataset(qa_dataset)
    bm25_eval_results = await bm25_retriever_evaluator.aevaluate_dataset(qa_dataset)
    hybrid_eval_results = await hybrid_retriever_evaluator.aevaluate_dataset(qa_dataset)

    print("\n✅ === Retrieval Metrics Comparison ===")
    display_retriever_eval_results("Base Retriever", base_eval_results)
    display_retriever_eval_results("Auto-Merging Retriever", auto_eval_results)
    display_retriever_eval_results("BM25 Retriever", bm25_eval_results)
    display_retriever_eval_results("Hybrid Retriever", hybrid_eval_results)

    # Display results table
    displayify_df(df)
    df.to_csv("results.csv", index=False)


# ✅ Execute Async Evaluation
asyncio.run(run_evaluation())

How does the document describe the relationships between chronic migraine and comorbidities?

Questions generated: How does the document describe the relationships between chronic migraine and comorbidities?

What are the comparative advantages and disadvantages of medical treatments and lifestyle modifications in chronic migraine management, as described in the document?

Questions generated: What are the comparative advantages and disadvantages of medical treatments and lifestyle modifications in chronic migraine management, as described in the document?

Can you list the side effects of VYEPTI detailed in this document?

Questions generated: Can you list the side effects of VYEPTI detailed in this document?

What monoclonal antibodies are approved for prophylactic migraine treatment?

Questions generated: What monoclonal antibodies are approved for prophylactic migraine treatment?

What emotional toll does the document portray as resulting from chronic migraine?

Questions generated

,Query,Base Response,Auto-Merged Response,BM25 Response,Hybrid Response,Base Context,Auto Context,BM25 Context,Hybrid Context,Base Faithfulness,Auto Faithfulness,BM25 Faithfulness,Hybrid Faithfulness,Base Relevancy,Auto Relevancy,BM25 Relevancy,Hybrid Relevancy
0,What are the connections between chronic migraine and other health conditions according to the document?,"The provided context does not explicitly state any connections between chronic migraine and other health conditions. It does mention that chronic migraine is a medical condition in itself, but it does not provide information about any specific links to other health conditions. It is always recommended to consult a healthcare provider for accurate and personalized information regarding health conditions and their potential connections.","According to the document, chronic migraine is often associated with several other health conditions. These include mood disorders such as depression and anxiety, sleep disorders like sleep apnea, insomnia, and restless legs syndrome, cardiovascular disorders, inflammatory conditions, neurological disorders like epilepsy, multiple sclerosis, and stroke, and other conditions that affect the brain. There is also a connection between chronic migraine and fibromyalgia, with over 55% of people with fibromyalgia also having chronic migraine. Additionally, there seems to be a connection between migraine and gastrointestinal issues, with irritable bowel syndrome being common. Lastly, there is an association between migraine and the development of inflammatory bowel disease in some patients.","According to the document, chronic migraine is associated with an increased risk of mood disorders such as depression and anxiety, sleep disorders like sleep apnea, insomnia, restless legs syndrome, and poor sleep quality and duration. There is also a relationship between chronic migraine and inflammatory conditions, particularly arthritis, although the exact link is not fully understood. People with chronic migraine are more likely to have one if they have the other. Additionally, there is a connection between chronic migraine and chronic pain disorders like fibromyalgia, with about 30% of people with chronic migraine also having fibromyalgia, and over 55% of people with fibromyalgia also having chronic migraine. Lastly, gastrointestinal issues such as nausea, diarrhea, vomiting, inflammatory bowel disease (IBD), and irritable bowel syndrome (IBS) are often seen in people with migraine.","According to the document, chronic migraine is associated with an increased risk of mood disorders such as depression and anxiety, sleep disorders like sleep apnea, insomnia, restless legs syndrome, poor sleep quality and duration, cardiovascular disorders such as heart attack, stroke, and death from cardiovascular disease, inflammatory conditions like arthritis, neurological disorders such as epilepsy, multiple sclerosis, and stroke, and gastrointestinal issues including nausea, diarrhea, vomiting, and disorders such as inflammatory bowel disease (IBD) and irritable bowel syndrome (IBS). Additionally, there is a relationship between chronic migraine and fibromyalgia, with about 30% of people with chronic migraine also having fibromyalgia, and over 55% of people with fibromyalgia also having chronic migraine.",chronic migraine contents news and more the latest research being my own advocate health smarts view... mething you have read in a publication if you think you have a medical emergency cain your doctor or,chronic migraine contents news and more the latest research being my own advocate health smarts view... e my best friend. THIS CONTENT IS CREATED AND CONTROLLED BY WEBMD’S EDITORIAL STAFF | WWW.WEBMD.COM,health smarts chronic migraine comorbidities these conditions often go handinhand with chronic migra... ased Study”; Association of Migraine Disorders: “The Relationship Between Migraine and Fibromyalgia”,# MOOD DISORDERS Your chance of having depression and 